In [1]:
%%time
import os
import pandas as pd
from datetime import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

CPU times: total: 812 ms
Wall time: 21.8 s


In [10]:
# Ruta de la carpeta que contiene los archivos
ruta_carpeta = 'data_files/'  # Reemplaza esto con la ruta de tu carpeta

# Lista para almacenar los DataFrames cargados
dataframes = []

# Recorre todos los archivos en la carpeta
for archivo in os.listdir(ruta_carpeta):
    # Verifica si el archivo tiene una extensión que corresponde a un archivo CSV o Excel
    if archivo.endswith('.csv') or archivo.endswith('.xlsx'):
        # Construye la ruta completa del archivo
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        #print("Cargando:", ruta_archivo)
        # Carga el archivo en un DataFrame y agrega a la lista (** hablar de la notación one line)
        df = pd.read_csv(ruta_archivo) if archivo.endswith('.csv') else pd.read_excel(ruta_archivo)
        #df.dropna(inplace=True)
        dataframes.append(df)
    
    #print(list(df.head(0).columns))
    #print(archivo,'\r\n',df.dtypes)
        
df_combinado = pd.concat(dataframes, ignore_index=True)
df_combinado.head(1)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"


In [3]:
def dropper(x):   
    x.dropna(inplace=True)
    return x

def clean_headers(x):   
    x = x.drop(x[x["Quantity Ordered"] == "Quantity Ordered"].index)
    return x

def formatear_fecha(df,col_name='Order Date'):
    df['fecha_objeto'] = pd.to_datetime(df[col_name], format="%m/%d/%y %H:%M")
    #df['date_fix'] = pd.to_datetime(df[col], format = "%m/%d/%y %H:%M")
    df['nueva_fecha'] = df['fecha_objeto'].dt.strftime("%d/%m/%Y")
    df['nuevo_tiempo'] = df['fecha_objeto'].dt.strftime("%H:%M")
    #df.drop(columns=['fecha_objeto'], inplace=True)
    return df

def convertir_formato_fecha(fecha_hora_str,col_name='Order Date'):
    formato_entrada = "%m/%d/%y %H:%M"
    fecha_hora_obj = datetime.strptime(fecha_hora_str, formato_entrada)    
    formato_salida = "%d/%m/%Y %H:%M" 
    fecha_nueva_str = fecha_hora_obj.strftime(formato_salida)   
    return fecha_nueva_str

def separar(cadena_texto):
    lista_salida = cadena_texto.split()    
    return lista_salida

def split_address(df, colname="Purchase Address"):
    splitted_list = df[colname].str.split(",")
    df["City"] = pd.DataFrame(splitted_list)[colname].str.get(1)
    df["State"] = pd.DataFrame(splitted_list)[colname].str.get(2)
    df["State"] = df["State"].str.extract(r'(\w+)')
    return df

def replace_float(df,colname="Price Each"):
    df[colname] = df[colname].astype(float)
    return df
def extract_mm_yy(df, colname="fecha_objeto"):
    df["month"] = df[df['fecha_objeto'].dt.year]
df_combinado = df_combinado.pipe(dropper).pipe(clean_headers).pipe(formatear_fecha).pipe(split_address).pipe(replace_float)


In [4]:
agrupacion_mes = pd.DataFrame(df_combinado.groupby([df_combinado['fecha_objeto'].dt.year, df_combinado['fecha_objeto'].dt.month])['Price Each'].sum())

In [5]:
agrupacion_mes

Price Each
fecha_objeto fecha_objeto            
2019         1             1803264.98
             2             2188884.72
             3             2791207.83
             4             3367671.02
             5             3135125.13
             6             2562025.61
             7             2632539.56
             8             2230345.42
             9             2084992.09
             10            3715554.83
             11            3180600.68
             12            4588415.41
2020         1                8503.40

#### P: ¿Cuál fue el mejor año en ventas? ¿Cuánto se ganó ese año?

In [12]:
agrupacion_mes = pd.DataFrame(df_combinado.groupby([df_combinado['fecha_objeto'].dt.year])['Price Each'].sum())
agrupacion_mes['Total_Sales'] = to_numeric(agrupacion_mes['Quantity Ordered'], errors='coerce')*agrupacion_mes['Price Each']

KeyError: 'fecha_objeto'

In [ ]:
El mejor año de ventas fue 2019 y se ganó 

#### P: ¿Cuál fue el mejor mes en ventas? ¿Cuánto se ganó ese mes?

#### P: ¿Qué ciudad tuvo el mayor número de ventas?

#### P: ¿A qué hora deberíamos mostrar publicidad para maximizar la probabilidad de que los clientes compren el producto?

#### P: ¿Qué productos se venden más a menudo juntos?

#### P: ¿Qué producto se vendió más? ¿Por qué crees que se vendió más?